# Proyecto Final: Sistema Multi-Agente de Documentación TI con Google ADK

Este cuaderno implementa una arquitectura de **Agentes Colaborativos** para transformar datos no estructurados en documentación técnica.

## Arquitectura "Doc Squad"
Simulamos un flujo de trabajo real con tres agentes especializados:

1.  **IngestAgent (El Bibliotecario)**: Gestiona la subida y validación de archivos multimedia.
2.  **AnalystAgent (El Ingeniero)**: Analiza el contenido técnico y extrae hechos puros (comandos, errores, topología).
3.  **TechWriterAgent (El Redactor)**: Toma los hechos y genera el documento final con formato y estilo profesional.

Utilizamos **Google ADK** y **Gemini 1.5 Pro**.

## 1. Instalación y Configuración

> **Nota de Buenas Prácticas (Entorno Local):** Si ejecutas este código en tu propia máquina, crea siempre un entorno virtual primero:
> ```bash
> python -m venv .venv
> source .venv/bin/activate
> ```
> En **Kaggle** o **Google Colab**, instalamos directamente en el kernel:


In [ ]:
!pip install -q google-adk google-generativeai

In [ ]:
import os
import time
import google.generativeai as genai
from google.adk.agents.llm_agent import Agent
from google.colab import userdata # O usar Kaggle Secrets

# --- CONFIGURACIÓN DE API KEY ---
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
except ImportError:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") or input("Introduce tu Google API Key: ")

genai.configure(api_key=GOOGLE_API_KEY)
print("API Key configurada.")

## 2. Herramientas del Sistema (Tools)

Definimos la herramienta de ingesta que usará el `IngestAgent`.

In [ ]:
def ingest_multimedia_tool(file_path: str) -> str:
    """
    Sube un archivo a la API de Gemini y espera a que esté listo.
    Retorna el URI del archivo o un mensaje de error.
    """
    if not os.path.exists(file_path):
        return f"ERROR: El archivo {file_path} no existe en el sistema local."

    print(f"[System] Subiendo {file_path}...")
    try:
        file_upload = genai.upload_file(file_path)
        
        while file_upload.state.name == "PROCESSING":
            print("[System] Procesando...", end=".", flush=True)
            time.sleep(2)
            file_upload = genai.get_file(file_upload.name)

        if file_upload.state.name == "FAILED":
            return "ERROR: Falló el procesamiento en Gemini."

        print(f"\n[System] Archivo listo: {file_upload.uri}")
        return file_upload.uri
    except Exception as e:
        return f"ERROR CRÍTICO: {str(e)}"

## 3. Definición de Agentes (The Doc Squad)

Creamos los tres agentes con instrucciones especializadas.

In [ ]:
# --- 1. INGEST AGENT ---
ingest_agent = Agent(
    model='gemini-1.5-flash-latest', # Modelo rápido y barato para tareas simples
    name='IngestAgent',
    description="Gestiona la carga de archivos.",
    instruction="""
    Eres el IngestAgent. Tu único trabajo es recibir rutas de archivos locales y subirlos usando la herramienta 'ingest_multimedia_tool'.
    Una vez tengas el URI, devuélvelo confirmando que está listo para análisis.
    Si la herramienta falla, reporta el error claramente.
    """,
    tools=[ingest_multimedia_tool]
)

# --- 2. ANALYST AGENT ---
analyst_agent = Agent(
    model='gemini-1.5-pro-latest', # Modelo potente para análisis profundo
    name='AnalystAgent',
    description="Analiza contenido técnico y extrae hechos.",
    instruction="""
    Eres el AnalystAgent, un Ingeniero de Sistemas Senior.
    Tu trabajo es recibir un URI de archivo (video, audio, imagen) y extraer TODOS los detalles técnicos.
    NO te preocupes por el formato bonito. Céntrate en la precisión.
    
    Debes extraer:
    - Comandos exactos ejecutados.
    - Mensajes de error o logs visibles.
    - Pasos de configuración realizados.
    - Direcciones IP, nombres de host, puertos.
    
    Salida esperada: Una lista de hechos técnicos crudos y cronológicos.
    """
)

# --- 3. TECH WRITER AGENT ---
tech_writer_agent = Agent(
    model='gemini-1.5-pro-latest',
    name='TechWriterAgent',
    description="Genera documentación final.",
    instruction="""
    Eres el TechWriterAgent. Recibes una lista de hechos técnicos de un analista.
    Tu trabajo es convertir esos hechos en un documento profesional (Markdown).
    
    Estructura requerida:
    1. Título Descriptivo.
    2. Resumen Ejecutivo (1 párrafo).
    3. Prerrequisitos (si los hay).
    4. Procedimiento Paso a Paso (numerado).
    5. Solución de Problemas (si aplica).
    
    Usa bloques de código para comandos. Añade notas de advertencia (WARNING) si ves algo peligroso.
    Tu tono debe ser formal, claro y directo.
    """
)

print("Squad inicializado: IngestAgent, AnalystAgent, TechWriterAgent.")

## 4. Orquestación del Flujo

Aquí definimos la función que coordina el paso de mensajes entre los agentes.

In [ ]:
def run_documentation_pipeline(file_path: str, request_context: str = ""):
    print(f"--- INICIANDO PIPELINE PARA: {file_path} ---\n")
    
    # PASO 1: INGESTA
    print("🤖 IngestAgent: Trabajando...")
    ingest_response = ingest_agent.query(f"Sube y procesa el archivo: {file_path}")
    file_uri = ingest_response.answer # Asumimos que el agente devuelve el URI en su respuesta
    # Nota: En un caso real, haríamos parsing más robusto para extraer el URI limpio si el agente habla mucho.
    # Para simplificar, asumimos que el agente incluye el URI en el texto y el siguiente agente es listo.
    print(f"✅ IngestAgent: {ingest_response.answer}\n")
    
    # PASO 2: ANÁLISIS
    print("🤖 AnalystAgent: Analizando contenido...")
    # Pasamos la respuesta del IngestAgent (que contiene el URI) al AnalystAgent
    analysis_prompt = f"Aquí tienes el resultado de la ingesta: {ingest_response.answer}. Contexto extra: {request_context}. Analiza los hechos técnicos."
    analysis_response = analyst_agent.query(analysis_prompt)
    print(f"✅ AnalystAgent: Hechos extraídos.\n")
    # print(analysis_response.answer) # Descomentar para debug

    # PASO 3: REDACCIÓN
    print("🤖 TechWriterAgent: Redactando documento final...")
    writer_prompt = f"Aquí tienes los hechos técnicos extraídos: \n{analysis_response.answer}\n. Genera el documento final."
    final_doc = tech_writer_agent.query(writer_prompt)
    
    return final_doc.answer

## 5. Demo en Vivo

Ejecutamos el pipeline completo.

In [ ]:
# Crear archivo dummy para la demo (si no existe)
dummy_filename = "demo_config_server.txt"
if not os.path.exists(dummy_filename):
    with open(dummy_filename, "w") as f:
        f.write("""
        [VIDEO TRANSCRIPT SIMULADO]
        Admin: Vamos a configurar el servidor web Apache.
        Admin: Primero, actualizamos los repositorios con 'sudo apt update'.
        Admin: Ahora instalamos apache con 'sudo apt install apache2 -y'.
        Admin: Verificamos el estado con 'systemctl status apache2'.
        Admin: Oh, veo un error en el log, el puerto 80 está ocupado.
        Admin: Vamos a cambiar el puerto en /etc/apache2/ports.conf a 8080.
        Admin: Reiniciamos el servicio 'sudo systemctl restart apache2'.
        Admin: Listo, funcionando en puerto 8080.
        """)

# Ejecutar Pipeline
final_documentation = run_documentation_pipeline(dummy_filename, request_context="Tutorial de instalación básico")

from IPython.display import Markdown
display(Markdown(final_documentation))